In [ ]:
import twint
from datetime import datetime

# For notebooks 
import nest_asyncio
nest_asyncio.apply()

In [ ]:

print(datetime.strptime("2019-04-29 00:00:00", '%Y-%m-%d %H:%M:%S'))
print(datetime.strptime("2021-04-29 12:00:00", '%Y-%m-%d %H:%M:%S'))

In [ ]:
#configuration
#configuration
config = twint.Config()
config.Search = "bitcoin"
config.Lang = "en"
config.Limit = 100
config.Since = "2019-04-29 00:00:00" #datetime.strptime("2019-04-29 00:00:00", '%Y-%m-%d %H:%M:%S')
config.Until = "2020-04-29 12:00:00" # datetime.strptime("2020–04–29 12:00:00", '%Y-%m-%d %H:%M:%S')
config.Min_likes = 500
config.Store_json = True
config.Output = "custom_out.json"

#running search
twint.run.Search(config)